In [1]:
import cv2
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
classes = ['CLOSE', 'FORWARD', 'LEFT', 'RIGHT']
model_DIRECTORY = 'model.hdf5'

#uses model from previous notebook
model = keras.models.load_model(model_DIRECTORY)

In [5]:
def img_process(img):
    img = cv2.resize(img, (100,100))
    img = tf.expand_dims(img, axis=0)
    return np.argmax(model(img))

In [6]:
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
#use haarcascade to isolate image to make predictions
video = cv2.VideoCapture(0)

while True:
    ret, frame = video.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=8, minSize=(30, 30))


    for (x, y, w, h) in eyes:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    if len(eyes):
        x, y, w, h = eyes[0]
        new_frame = frame[y:y+h, x:x+w]
        result = img_process(new_frame)
        cv2.putText(frame, f'{classes[result]}', (20,40), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 2)

    cv2.imshow('Eyes Detected', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()
